In [2]:
import pandas as pd
from langchain.chains import LLMChain, SequentialChain


In [3]:
import prompts as p


DATA_PATH = '../datasets/downsampled_50_50.csv'

# Load data
df = pd.read_csv(DATA_PATH)

df.fillna('', inplace=True)

prompt = p.prompt0_en_baseline

input_variables=['p1', 'p2'] output_parser=None partial_variables={} template='Could you please tell me if the following two products are the same one?\nProduct 1: {p1}\nProduct 2: {p2}\n\nPlease answer my question only with True or False.' template_format='f-string' validate_template=True


In [72]:
df.head(30)

,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,cluster_id_left,id_right,brand_right,title_right,description_right,price_right,priceCurrency_right,cluster_id_right,pair_id,label,is_hard_negative,prompt_1_gpt-3.5-turbo_temp-0,prompt_2_gpt-3.5-turbo_temp-0,prompt_3_gpt-3.5-turbo_temp-0
0,24080197,,DJI Mavic Mini Drone Fly More Combo,"Mavic Mini, the everyday flycam. Open your sky...",649.00,CAD,1709252,92817369,Fujifilm,Fujifilm Instax Mini Instant Film - Monochrome...,Monochrome (Black & White) Instant Film for Fu...,159,ZAR,165928,24080197#92817369,False,True,False,False,False
1,94788371,,Instax Mini Rainbow Film 10 Pack,Brighten the place up with this pack of rainbo...,14.99,GBP,1374090,10718715,,1 Fujifilm Instax Square Film Black Frame,,3800,HUF,1118780,94788371#10718715,False,True,False,False,False
2,14016048,,Blackmagic Design ATEM Production Studio 4K Sw...,Blackmagic Design ATEM Production Studio 4K Sw...,1.695E3,USD,1263992,52586114,,"Western Digital Blue PC 2.5\"" 250 GB Serial AT...","250 GB, 2.5\"", SATA 6Gb/s, 540/500 MB/s",62.436001,EUR,483248,14016048#52586114,False,False,False,False,False
3,62248973,Instax,Instax Fujifilm | Instax SQUARE Instant Film -...,,14.99,CAD,1118780,63595157,,[Fujifilm]100 Sheet Fujifilm Instax Square Pla...,100 sheet instax Square Film,98.50,USD,300521,62248973#63595157,False,True,False,False,False
4,25011125,,Hikvision DVR 16 CH fino a 8Mpx Turbo 3.0 DS-7...,Hikvision DVR 16 CH fino a 8Mpx Turbo 3.0 DS-7...,3.726E2,EUR,1336313,10718715,,1 Fujifilm Instax Square Film Black Frame,,3800,HUF,1118780,25011125#10718715,False,False,False,False,False
5,60568221,,TAG Heuer Formula 1 Chronograph 43mm CAZ1010.F...,,1169.75,EUR,2107019,31469161,Sony,Sony 30mm f/3.5 Macro Lens,Offers excellent optical sharpness and clarity...,499,AUD,213496,60568221#31469161,False,True,False,False,False
6,27700752,,SanDisk Extreme PRO SD UHS-II USB 3.0,,99,PLN,137095,86534825,SanDisk,SanDisk Extreme Pro CFAST 2.0 VPG130 - 64GB,For those working in high-performance industri...,310.00,SGD,1013997,27700752#86534825,False,True,False,False,False
7,64446755,Brother,"TN-200 sort lasertoner, original Brother TN200...",Ny original Brother sort tonerpatron til Broth...,339.00,DKK,5223239,65569829,,MINION DHR II 29 X 2.30 3C TERRA TR DD FOLD,Refined and redesigned; ready to shred any lin...,1.0999E2,AUD,1084991,64446755#65569829,False,False,False,False,False
8,54979311,,TP-Link TL-WN781ND Netcard PCI-eX Wireless 150 mb,,,,466493,34393388,,Adattatore USB Wireless TP-Link TL-WN725N 150M...,,6,EUR,97189,54979311#34393388,False,True,False,False,False
9,89974024,,Refill #014: Kraft Paper Notebook,Cardboard Paper notebook refill for the Travel...,7.9,EUR,1857459,58146758,,"Brother TN-241M (prentar:1,400 síður) rautt du...",Nánari lýsing frá framleiðanda:Barcode: 497776...,16072,ISK,499089,89974024#58146758,False,True,False,False,False


In [73]:
from langchain.chat_models import ChatOpenAI


In [74]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [75]:
from typing import Optional, Union
from langfuse.callback import CallbackHandler
handler = CallbackHandler(os.environ.get("LF_PK"), os.environ.get("LF_SK"), os.environ.get("LF_HOST"))

In [76]:


prompt = PromptTemplate(
    input_variables=["p1", "p2"],
    template="""Your task is to decide if two product descriptions refer to the same product.

    Think step by step!
    Product 1: {p2}
    Product 2: {p1}
"""
)

evalPrompt = PromptTemplate(
    input_variables=["chain_of_thought_reasoning"],
    template="""The following text is the answer and reasoning if two product descriptions refer to the same product. 
    Please evaluate if the the overall conclusion is that the two products match or not.
    Your answer should be either "match" or "no match".
    Answer: {chain_of_thought_reasoning}
"""
)




In [77]:
def cut_length(string ,length = 250):
    return string[0:length]

In [78]:

# product1 = "Title: " + cut_length(str(df["title_left"][1] or "none")) + "\nDescription: " + cut_length(str(df["description_left"][1] or "none"))
# product2 = "Title: " + cut_length(str(df["title_right"][1] or "none")) + "\nDescription: " + cut_length(str(df["description_right"][1] or "none"))



# product1 = "Title: " + cut_length(str(df["title_left"][1] or "none")) + "\nDescription: " + cut_length(str(df["description_left"][1] or "none")) + "\nBrand:" + str(df["brand_left"][1] or "none") 
# product2 = "Title: " + cut_length(str(df["title_right"][1] or "none")) + "\nDescription: " + cut_length(str(df["description_right"][1] or "none")) + "\nBrand:" + str(df["brand_right"][1] or "none") 

In [79]:
# def evaluate(df, prompt, temp=0, start = 0):
#     llm = ChatOpenAI(model_name= "gpt-3.5-turbo", temperature=temp)
#     for i in range(start, len(df)):
#         print(i)
#         # Creating Prompot and sending request and printing informations
        
#         product1 = "Title: " + cut_length(str(df["title_left"][i] or "none")) + "\nDescription: " + cut_length(str(df["description_left"][i] or "none")) + "\nBrand:" + str(df["brand_left"][i] or "none")
#         product2 = "Title: " + cut_length(str(df["title_right"][i] or "none")) + "\nDescription: " + cut_length(str(df["description_right"][i] or "none")) + "\nBrand:" + str(df["brand_right"][i] or "none")
        
#         chain = LLMChain(llm=llm, prompt=prompt)

        
#         response = chain.run(p1=product1, p2=product2)

#         # result = response["choices"][0]["text"]
        
        
#         result = response

#         if(i < 5):
#             print(chain.prompt)
#             print("__\n__")
#             print(response)
#             print("__\n__")
        
#         #Processing Response
#         isMatch = False
#         if  "Yes" in result:
#             isMatch = True
#         elif ("No" in result):
#             isMatch = False
        
#         print("isMatch", str(isMatch))
#         df["prompt_3_gpt-3.5-turbo_temp-0"][i] = isMatch
#         # saveResponse(prompt, response, isMatch)

In [80]:
def evaluate(df, prompt, temp=0, start = 0):
    llm1 = ChatOpenAI(model_name= "gpt-3.5-turbo", temperature=temp)
    llm2 = ChatOpenAI(model_name= "gpt-3.5-turbo", temperature=temp, max_tokens=10)
    for i in range(start, len(df)):
        print(i)
        # Creating Prompot and sending request and printing informations
        
        product1 = "Title: " + cut_length(str(df["title_left"][i] or "none")) + "\nDescription: " + cut_length(str(df["description_left"][i] or "none")) + "\nBrand:" + str(df["brand_left"][i] or "none")
        product2 = "Title: " + cut_length(str(df["title_right"][i] or "none")) + "\nDescription: " + cut_length(str(df["description_right"][i] or "none")) + "\nBrand:" + str(df["brand_right"][i] or "none")
        
        chain = LLMChain(llm=llm1, prompt=prompt, output_key="chain_of_thought_reasoning")

        eval_chain = LLMChain(llm=llm2, prompt=evalPrompt, output_key="match_result")

        overall_chain = SequentialChain(chains=[chain, eval_chain], input_variables=["p1", "p2"], output_variables=["chain_of_thought_reasoning", "match_result"], verbose=True)
        
        response = overall_chain({"p1":product1, "p2":product2}, callbacks=[handler])

        handler.langfuse.flush()

        # result = response["choices"][0]["text"]
        
        
        result = response

        # if(i < 5):
        print(chain.prompt)
        print("__\n__")
        print(response)
        print("__\n__")
        
        #Processing Response
        isMatch = False
        if  "no" in result:
            isMatch = False
        elif ("match" in result):
            isMatch = True
        
        print("isMatch", str(isMatch))
        df["prompt_3_gpt-3.5-turbo_temp-0"][i] = isMatch
        # saveResponse(prompt, response, isMatch)

In [81]:
evaluate(df, prompt,temp=0, start=0)


0


> Entering new SequentialChain chain...

> Finished chain.
input_variables=['p1', 'p2'] output_parser=None partial_variables={} template='Your task is to decide if two product descriptions refer to the same product.\n\n    Think step by step!\n    Product 1: {p2}\n    Product 2: {p1}\n' template_format='f-string' validate_template=True
__
__
{'p1': "Title: DJI Mavic Mini Drone Fly More Combo\nDescription: Mavic Mini, the everyday flycam. Open your sky with the compact, yet powerful Mavic Mini. Weighing under 250 grams, together with the DJI Fly APP, you'll enjoy a simplified flying experience that is almost as light as the average smartphone.\nBrand:none", 'p2': 'Title: Fujifilm Instax Mini Instant Film - Monochrome (10 Shots)\nDescription: Monochrome (Black & White) Instant Film for Fujifilm Mini Instant Cameras10 Shots per PackISO 800High Gloss FinishSuperb Grain QualityFast Image FormationGreater Resistance to Age-Induced ChangesSingle-plastic Composition for Easier Disposal\nBr

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_2910/2371505386.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["prompt_3_gpt-3.5-turbo_temp-0"][i] = isMatch



> Finished chain.
input_variables=['p1', 'p2'] output_parser=None partial_variables={} template='Your task is to decide if two product descriptions refer to the same product.\n\n    Think step by step!\n    Product 1: {p2}\n    Product 2: {p1}\n' template_format='f-string' validate_template=True
__
__
{'p1': 'Title: Instax Mini Rainbow Film 10 Pack\nDescription: Brighten the place up with this pack of rainbow coloured film! Top tip: one of these would look fab in our Instax Mini Acrylic Photo Block!Includes pack of 10 shots\nBrand:none', 'p2': 'Title: 1 Fujifilm Instax Square Film Black Frame\nDescription: none\nBrand:none', 'chain_of_thought_reasoning': 'Step 1: Compare the titles of the two products.\n- Product 1: "1 Fujifilm Instax Square Film Black Frame"\n- Product 2: "Instax Mini Rainbow Film 10 Pack"\n\nThe titles indicate that these are different products. Product 1 is a Fujifilm Instax Square Film with a black frame, while Product 2 is an Instax Mini Rainbow Film in a 10 pack

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_2910/2371505386.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["prompt_3_gpt-3.5-turbo_temp-0"][i] = isMatch



> Finished chain.
input_variables=['p1', 'p2'] output_parser=None partial_variables={} template='Your task is to decide if two product descriptions refer to the same product.\n\n    Think step by step!\n    Product 1: {p2}\n    Product 2: {p1}\n' template_format='f-string' validate_template=True
__
__
{'p1': 'Title: Blackmagic Design ATEM Production Studio 4K Switcher\nDescription: Blackmagic Design ATEM Production Studio 4K SwitchATEM Production Studio 4K lets you connect up to 8 SD, HD or Ultra HD 4K video cameras, disk recorders and computers to do true broadcast quality work. You get all the features you expect from a profe\nBrand:none', 'p2': 'Title: Western Digital Blue PC 2.5\\" 250 GB Serial ATA III\nDescription: 250 GB, 2.5\\", SATA 6Gb/s, 540/500 MB/s\nBrand:none', 'chain_of_thought_reasoning': 'Based on the given information, it is not possible to determine if the two product descriptions refer to the same product. The descriptions provide different details about the produc

...


> Finished chain.
input_variables=['p1', 'p2'] output_parser=None partial_variables={} template='Your task is to decide if two product descriptions refer to the same product.\n\n    Think step by step!\n    Product 1: {p2}\n    Product 2: {p1}\n' template_format='f-string' validate_template=True
__
__
{'p1': 'Title: TP-Link TL-WN781ND Netcard PCI-eX Wireless 150 mb\nDescription: none\nBrand:none', 'p2': 'Title: Adattatore USB Wireless TP-Link TL-WN725N 150Mbps 2.4GHz\nDescription: none\nBrand:none', 'chain_of_thought_reasoning': 'Step 1: Compare the titles of the two product descriptions.\n\nThe titles of the two product descriptions are:\n1. Adattatore USB Wireless TP-Link TL-WN725N 150Mbps 2.4GHz\n2. TP-Link TL-WN781ND Netcard PCI-eX Wireless 150 mb\n\nStep 2: Analyze the titles to identify any similarities or differences.\n\nBoth titles mention TP-Link, which indicates that both products are from the same brand. Additionally, both titles mention "Wireless" and "150" in their descri

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_2910/2371505386.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["prompt_3_gpt-3.5-turbo_temp-0"][i] = isMatch



> Finished chain.
input_variables=['p1', 'p2'] output_parser=None partial_variables={} template='Your task is to decide if two product descriptions refer to the same product.\n\n    Think step by step!\n    Product 1: {p2}\n    Product 2: {p1}\n' template_format='f-string' validate_template=True
__
__
{'p1': "Title: Refill #014: Kraft Paper Notebook\nDescription: Cardboard Paper notebook refill for the Traveler's Notebook.Various cuttings, stickers, photos, scribbles and more sophisticated drawings suit it well.Contains 64 pages and its dimensions (210 x 110 mm) make it suitable for the classic version, not f\nBrand:none", 'p2': 'Title: Brother TN-241M (prentar:1,400 síður) rautt dufthylki\nDescription: Nánari lýsing frá framleiðanda:Barcode: 4977766718424Compatible Brand: BrotherCompatible Printer(s): HL-3140CW • HL-3150CDW • HL-3170CDW • DCP-9020CDW • MFC-9140CDN • MFC9330CDW • MFC9340CDWEC Commodity Code: 84439990Form Factor: CartridgeModel Name:\nBrand:none', 'chain_of_thought_rea

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_2910/2371505386.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["prompt_3_gpt-3.5-turbo_temp-0"][i] = isMatch


KeyboardInterrupt: 

In [ ]:
# write df to csv
df.to_csv(DATA_PATH, index=False)